<a href="https://colab.research.google.com/github/flee2-trinity/Deep-Learning-in-Sports-Result-Prediction-/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from google.colab import drive

drive.mount('/content/gdrive')
DATA_PATH = '/content/gdrive/My Drive/2021 Fall/Thesis I'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import datetime

def findFiles(path): 
  return glob.glob(path)

In [ ]:
import csv

filename = '/content/gdrive/My Drive/2021 Fall/Thesis I/NBA Data/nba 18-19/NatStat-NBA2019-Team_Statlines.csv'
with open(filename, newline='') as f:
      reader = csv.reader(f)
      """
      print(list(reader))
      ['GameDay', 'GameID', 'TeamID', 'Team', 'OpponentID', 'Opponent', 'Location',
       'Division', 'Conference', 'Playoffs', 'WinOrLoss', 'PlayerType', 'MIN', 'PTS', 
       'FGM', 'FGA', '3FM', '3FA', 'FTM', 'FTA', 'REB', 'AST', 'STL', 'BLK', 'OREB', 'TO', 'PF']
      """
      next(reader)
      rawDataL = list(reader)
print(rawDataL[0])

['2018-10-16', '985248', '5', 'Boston', '28', 'Philadelphia', 'H', '', '', '', 'W', '', '234', '103', '41', '96', '11', '37', '10', '14', '54', '21', '7', '6', '12', '14', '20']


In [ ]:
import datetime

def readTime(ampm):
  t = 0
  tL = ampm.replace(' ',':').split(':')
  if tL[2] == "PM" and int(tL[0]) < 12:
    t += 12
  t += int(tL[0])
  t += int(tL[1])/60
  return t

seasonBegin = datetime.datetime(2018, 10, 16)
noonTime = readTime("12:00 PM")
#not using
class Stats:
  def __init__(self):
    """
    MIN, PTS,FGM, FGA, 3FM, 3FA, FTM, FTA, REB, AST, STL, BLK, OREB, TO, PF
    """
    self.MIN = 0
    self.PTS = 0
    self.FGM = 0
    self.FGA = 0
    self.FM3 = 0
    self.FA3 = 0
    self.FTM = 0
    self.FTA = 0
    self.REB = 0
    self.AST = 0
    self.STL = 0
    self.BLK = 0
    self.OREB = 0
    self.TO = 0
    self.PF = 0
    self.Win = 0
    self.Loss = 0
    self.WinRate = 0
    self.homeCount = 0
    self.gameCount = 0
    self.firstGame = False
  def update(self, MIN, PTS,FGM, FGA, FM3, FA3, FTM, FTA, REB, AST, STL, BLK, OREB, TO, PF):
    self.MIN = MIN
    self.PTS = PTS
    self.FGM = FGM
    self.FGA = FGA
    self.FM3 = FM3
    self.FA3 = FA3
    self.FTM = FTM
    self.FTA = FTA
    self.REB = REB
    self.AST = AST
    self.STL = STL
    self.BLK = BLK
    self.OREB = OREB
    self.TO = TO
    self.PF = PF
  def itsFirstGame(self):
    self.firstGame = True
  def addWinRate(self, Win, Loss, homeC):
    self.homeCount = homeC
    self.Win = Win
    self.Loss = Loss
    self.gameCount = Win+Loss
    self.WinRate = Win/(Win+Loss)
    self.homeRate = homeC / (Win+Loss)
  def notEmpty(self):
    return MIN>0
class aGame:
  def __init__(self, GameDay, GameID, TeamID, Team, OpponentID, Opponent, Location,
       Division, Conference, Playoffs, WinOrLoss, PlayerType):
    self.GameID = int(GameID)
    self.TeamID = int(TeamID)
    gdL = GameDay.split("-")
    self.GameDay = datetime.datetime(int(gdL[0]), int(gdL[1]), int(gdL[2]))
    #self.GameTime = GameTime
    self.DayCount = (self.GameDay - seasonBegin).days
    self.Team = Team
    self.OpponentID = int(OpponentID)
    self.Opponent = Opponent
    self.Home = Location == 'H'
    self.HomeID = int( Location == 'H')
    self.Division = Division
    self.Conference = Conference
    self.Playoffs = Playoffs == 'Y'
    self.WinOrLoss = WinOrLoss == 'W'
    self.WinID = int(self.WinOrLoss)
    self.LossID = int(not self.WinOrLoss)
    self.PlayerType = PlayerType
    self.TStats = Stats()
    self.OppStats = Stats()
    self.TStatsP = Stats()
    self.OppStatsP = Stats()
    #self.TRecords = winRecords(TeamID, Team)
    #self.OppRecords = winRecords(OpponentID, Opponent)
  def printDay(self):
    print('Days', self.DayCount)

def aLinetoGame(aline):
  newG = aGame(aline[0], aline[1], aline[2], aline[3], aline[4], aline[5], aline[6],
               aline[7], aline[8], aline[9], aline[10], aline[11])
  for n in range(12, 27):
    aline[n] = int(aline[n])

  newG.TStats.update(aline[12], aline[13], aline[14], aline[15], aline[16], aline[17], 
                     aline[18], aline[19], aline[20], aline[21], aline[22], aline[23], 
                     aline[24], aline[25], aline[26])
  return newG

def readLines(filename):
  with open(filename, newline='') as f:
      reader = csv.reader(f)
      next(reader)
      rawDataL = list(reader)
  #print(rawDataL)
  #'ID', 'GameDay', 'GameTime', 'Home', 'HomeID', 'Visitor', 'VisitorID', 'ScoreVis', 
  #'ScoreHome', 'GameStatus', 'Venue', 'CityState', 'Neutral', 'Division', 'Conference', 'Playoffs'
  #print("RawDataLength ",len(rawDataL))
  data = []
  for oneLine in rawDataL:
    g = aLinetoGame(oneLine)
    data.append(g)
  return data

In [ ]:
from copy import deepcopy
allData = readLines(filename)
print(allData[1])
print(len(allData))

for tg in allData:
  previousGamesList = list(filter(lambda g: (g.DayCount < tg.DayCount) and (g.TeamID == tg.TeamID), allData))
  pMIN = 0
  pPTS = 0
  pFGM = 0
  pFGA = 0
  pFM3 = 0
  pFA3 = 0
  pFTM = 0
  pFTA = 0
  pREB = 0
  pAST = 0
  pSTL = 0
  pBLK = 0
  pOREB = 0
  pTO = 0
  pPF = 0
  pWin = 0
  pLoss = 0
  pHomeCount = 0
  for p in previousGamesList:
    g = p.TStats
    pMIN += g.MIN
    pPTS += g.PTS
    pFGM += g.FGM
    pFGA += g.FGA
    pFM3 += g.FM3
    pFA3 += g.FA3
    pFTM += g.FTM
    pFTA += g.FTA
    pREB += g.REB
    pAST += g.AST
    pSTL += g.STL
    pBLK += g.BLK
    pOREB += g.OREB
    pTO += g.TO
    pPF += g.PF
    pWin += p.WinID
    pLoss += p.LossID
    pHomeCount += p.HomeID
  gPlayed = len(previousGamesList)
  if gPlayed > 0:
    tg.TStatsP.update(pMIN/gPlayed, pPTS/gPlayed, pFGM/gPlayed, pFGA/gPlayed, pFM3/gPlayed,
                      pFA3/gPlayed, pFTM/gPlayed, pFTA/gPlayed, pREB/gPlayed, pAST/gPlayed,
                      pSTL/gPlayed, pBLK/gPlayed, pOREB/gPlayed, pTO/gPlayed, pPF/gPlayed)
    tg.TStatsP.addWinRate(pWin, pLoss, pHomeCount)
  else:
    tg.TStatsP.update(234, 100, 44, 90, 10,
                      30, 20, 25, 45, 25,
                      7, 7, 11, 15, 22)
    tg.TStatsP.addWinRate(1, 1, 1)
    tg.TStatsP.itsFirstGame()
    #Random for the first game
everyGame = allData

for d in allData:
  #print(d.TStats.Win)
  sameGameList = list(filter(lambda g: g.GameID == d.GameID and g.TeamID != d.TeamID, allData))
  #Same game Opponent Stats
  if len(sameGameList) != 1:
    print("error here")
  g = sameGameList[0]
  #print(g.TStats.Win)
  d.OppStats = deepcopy(g.TStats)
  d.OppStatsP = deepcopy(g.TStatsP)
  g.OppStats = deepcopy(d.TStats)
  g.OppStatsP = deepcopy(d.TStatsP)
  #print(g.TStats.Win)
  """
allGames = []
for d in allData:
  sameGameList = list(filter(lambda g: g.GameID == d.GameID, allGames))
  if len(sameGameList) == 1:
    for g in sameGameList:
      if g.Home: 
        g.OppStats = deepcopy(d.TStats)
        g.OppStatsP = deepcopy(d.TStatsP)
      else:
        d.OppStats = deepcopy(g.TStats)
        d.OppStatsP = deepcopy(g.TStatsP)
        allGames.remove(g)
        allGames.append(d)
  else:
    allGames.append(d)
allData = allGames
    """

print(len(allData))

2580
2580


In [ ]:
#everyGame
def setGames(games, teams):
  setList = []
  for t in teams:
    s = list(filter(lambda g: g.TeamID == t, games))
    setList.append((sorted(s, key = lambda g : g.DayCount), t))
  return setList
teamList = list({g.TeamID for g in allData})
teamGames = setGames(allData, teamList)
teamN = len(teamList)

In [ ]:
import random
def randomPick(allData):
  afterN = 120
  testDaysN = 18
  allData = list(filter(lambda g:not g.Playoffs, allData))
  nData = len(allData)
  test_daysAfterN = list(filter(lambda g: g.DayCount >= afterN, allData))
  test_daysAfterN = list(map(lambda g: g.DayCount, test_daysAfterN))
  test_days = random.sample(test_daysAfterN, testDaysN)
  test_days.sort()
  test_list = list(filter(lambda g: g.DayCount in test_days and g.Home, allData))
  test_listGroups = []
  for d in test_days:
    oneGroup = []
    for g in allData:
      if g.DayCount == d and g.Home:
        oneGroup.append(g)
    test_listGroups.append(oneGroup)

  #print(len(allData))
  #print(len(test_list))
  #print(len(test_listGroups))
  test_list = sorted(test_list, key= lambda g: g.DayCount)
  train_listGroups = []
  for d in test_days:
    gs = list(filter(lambda g: d - g.DayCount > 0 and g.Home, allData))
    train_listGroups.append(gs)
  #print(len(train_listGroups))
  homeWinSum1 = list(filter(lambda g: g.WinOrLoss, test_list))
  print("Test: %f" %(len(homeWinSum1)/len(test_list)))
  return (test_listGroups, train_listGroups, testDaysN)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

tensor_dim = 25
class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(Net,self).__init__()
    self.hidden_size = hidden_size
    self.input_size = input_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
    self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
    self.i2o = nn.Linear(input_size + hidden_size, hidden_size)
    #self.dropout = nn.Dropout(drop_prob)
    self.fc1 = nn.Linear(hidden_size, 16)
    self.act1 = nn.Tanh()
    #self.fc2 = nn.Linear(64, 16)
    #self.act2 = nn.Tanh()
    self.fc3 = nn.Linear(16, 2)
    #self.soft = nn.LogSoftmax(dim=1)
    #self.soft = nn.LogSoftmax(dim=1)
    self.sig = nn.Sigmoid()
    #self.relu = nn.ReLU()
  def forward(self, input, hidden):
    #out, hidden = self.lstm(input, hidden)
    combined = torch.cat((input, hidden), dim=2)
    hidden = self.i2h(combined)
    out = self.i2o(combined)
    out = self.fc1(out)
    out = self.act1(out)
    #out = self.fc2(out)
    #out = self.act2(out)
    out = self.fc3(out)
    out = self.sig(out)
    #out = self.relu(out)
    #out = self.soft(out)
    #out = self.m(out)
    #out = torch.flatten(out)
    return out, hidden 
  def init_hidden(self):
        return torch.zeros(self.num_layers, 1, self.hidden_size)




In [ ]:
#teamGames
def twoGamesTensor(homeG, oppG, g):
  tensor = torch.zeros(1, 1, tensor_dim)
  tensor[0][0][0] = g.DayCount/300
  tensor[0][0][1] = g.TStatsP.Win/72
  tensor[0][0][2] = g.TStatsP.Loss/72
  tensor[0][0][3] = g.TStatsP.WinRate
  tensor[0][0][4] = g.OppStatsP.Win/72
  tensor[0][0][5] = g.OppStatsP.Loss/72
  tensor[0][0][6] = g.OppStatsP.WinRate
  #tensor[0][0][teamList.index(g.TeamID)] = 1
  #tensor[0][0][teamList.index(g.OpponentID) + teamN] = 1

  tensor[0][0][7] = homeG.HomeID
  tensor[0][0][8] = homeG.TStatsP.Win/72
  tensor[0][0][9] = homeG.TStatsP.Loss/72
  tensor[0][0][10] = homeG.TStatsP.WinRate
  tensor[0][0][11] = homeG.OppStatsP.Win/72
  tensor[0][0][12] = homeG.OppStatsP.Loss/72
  tensor[0][0][13] = homeG.OppStatsP.WinRate
  tensor[0][0][14] = homeG.WinID
  tensor[0][0][15] = oppG.HomeID
  tensor[0][0][16] = oppG.TStatsP.Win/72
  tensor[0][0][17] = oppG.TStatsP.Loss/72
  tensor[0][0][18] = oppG.TStatsP.WinRate
  tensor[0][0][19] = oppG.OppStatsP.Win/72
  tensor[0][0][20] = oppG.OppStatsP.Loss/72
  tensor[0][0][21] = oppG.OppStatsP.WinRate
  tensor[0][0][22] = oppG.WinID
  tensor[0][0][23] = homeG.DayCount/300
  tensor[0][0][24] = oppG.DayCount/300
  return tensor
def merge2Mgames(tlist, tlen, olist, olen, N, game):
  d = min(tlen, olen)
  d = min(d, N)
  tempList = []
  for i in range(d):
    tempList.append(twoGamesTensor(tlist[tlen - d + i], olist[olen - d + i], game))
  labelTensor = torch.zeros(1, 1, 2)
  if game.WinID > 0:
    labelTensor[0][0][1] = 1
  else:
    labelTensor[0][0][0] = 1
  return (tempList, labelTensor)
def getNpreviousGame(listGroup, N):
  NpreviousGame = []
  for g in listGroup:
    pDay = g.DayCount
    tlist, _ = list(filter(lambda t: g.TeamID == t[1], teamGames))[0]
    tlist = list(filter(lambda h :h.DayCount < pDay, tlist))
    tlist = sorted(tlist, key = lambda h : h.DayCount)
    tIndex = len(tlist)
    olist, _ = list(filter(lambda t: g.OpponentID == t[1], teamGames))[0]
    olist = list(filter(lambda h :h.DayCount < pDay, olist))
    olist = sorted(olist, key = lambda h : h.DayCount)
    oIndex = len(olist)
    NpreviousGame.append(merge2Mgames(tlist, tIndex, olist, oIndex, N, g))
  return NpreviousGame
def getLNpreviousGame(listGroups, N):
  return list(map(lambda group: getNpreviousGame(group, N), listGroups))
  """
a,b, c = randomPick(allData)
trainlist = getLNpreviousGame(b, 15)
print(len(b))
print(len(trainlist))
print(len(trainlist[0]))
"""

In [ ]:
import time
import math
from statistics import mean
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def runModel(test_listGroups, train_listGroups, testDaysN, N):
  start = time.time()
  #test_tlistGroups = list(map(lambda group: gamestoTensor(group, test = True), test_listGroups))
  #train_tlist = gamestoTensor(train_list)
  #train_tlistGroups = list(map(lambda group: gamestoTensor(group), train_listGroups))
  train_tlistGroups = getLNpreviousGame(train_listGroups, N)
  test_tlistGroups = getLNpreviousGame(test_listGroups, N)
  correctTotal = 0
  totalGames = 0
  print(len(train_tlistGroups), len(test_tlistGroups))
  for t in range(testDaysN):
    print('Model: ', t)
    torch.autograd.set_detect_anomaly(True)
    model = Net(tensor_dim, 8, 1)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.005)
    loss_fn = nn.BCELoss()#meansquarerror
    n_epochs = 50
    for epoch in range(n_epochs):
      
      #train_loader = torch.utils.data.DataLoader(train_tlistGroups[t], batch_size = 1, shuffle = True)
      for gs, label in train_tlistGroups[t]:
        if len(gs) > 0:
          #newhidden = model.init_hidden() 
          hidden = model.init_hidden()
          outputs = 0
          for g in gs:
            
            
            outputs, hidden = model(g, hidden)
            #outputs, hidden = model(g, newhidden)
            #print(outputs)
            #newhidden = (torch.clone(hidden[0]), torch.clone(hidden[1]))

          optimizer.zero_grad()
          loss = loss_fn(outputs, label)
          loss.backward(retain_graph=True)
          optimizer.step()
            
        
      if (epoch+1)%10 == 0:
        print(timeSince(start))
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
        
    correct1 = 0
    totalGN = 0
    for gs, label in test_tlistGroups[t]:
        if len(gs) > 0:
          hidden = model.init_hidden() 
          outputTemp = 0
          for g in gs:
            outputs, hidden = model(g, hidden)
            outputTemp = outputs
            #print(outputs)
          predicted = torch.argmax(outputTemp, keepdim=False)
          labelIndex = torch.argmax(label, keepdim=False)
          totalGN += 1
          correct1 += int(torch.equal(predicted, labelIndex))
    accu = correct1 / totalGN
    correctTotal += correct1
    totalGames += totalGN
    print("Accuracy: %f" % (accu))
  totalAccu = correctTotal/totalGames
  print ("Overall Accuracy: %f" %(totalAccu))
  print(timeSince(start))
  return totalAccu

In [ ]:

def writeIn(newRecord):
  p1Name='/content/gdrive/My Drive/2021 Fall/Thesis I/Records/RLSTM_winloss_10_11.txt'

  p1file = open(p1Name)
  p1content = p1file.read()
  p1file_content = p1file.read()
  p1file.seek(0)
  #Set file’s current position to the beginning
  p1file=open(p1Name,"w")
  p1file.write(p1content+str(newRecord)+"\n")
  p1file=open(p1Name)
  p1file.seek(0)
  p1file_content = p1file.read()
  p1file.close()

  print(p1file_content)

In [ ]:
for i in range(3):
  a,b, c = randomPick(allData)
  writeIn(runModel(a,b,c, 3))

Test: 0.591241
18 18
Model:  0
2m 47s
Epoch: 9, Loss: 0.710492
5m 24s
Epoch: 19, Loss: 0.827404
8m 2s
Epoch: 29, Loss: 0.901149
10m 39s
Epoch: 39, Loss: 0.961028
13m 16s
Epoch: 49, Loss: 0.991999
Accuracy: 0.800000
Model:  1
15m 58s
Epoch: 9, Loss: 0.336560
18m 41s
Epoch: 19, Loss: 0.228924
21m 22s
Epoch: 29, Loss: 0.229606
24m 4s
Epoch: 39, Loss: 0.231832
26m 48s
Epoch: 49, Loss: 0.234610
Accuracy: 0.363636
Model:  2
29m 32s
Epoch: 9, Loss: 0.348115
32m 15s
Epoch: 19, Loss: 0.216360
34m 58s
Epoch: 29, Loss: 0.209890
37m 41s
Epoch: 39, Loss: 0.209228
40m 23s
Epoch: 49, Loss: 0.210378
Accuracy: 0.363636
Model:  3
43m 12s
Epoch: 9, Loss: 0.697187
46m 1s
Epoch: 19, Loss: 0.789719
48m 49s
Epoch: 29, Loss: 0.754935
51m 37s
Epoch: 39, Loss: 0.724776
54m 27s
Epoch: 49, Loss: 0.710711
Accuracy: 0.454545
Model:  4
57m 23s
Epoch: 9, Loss: 0.731271
60m 19s
Epoch: 19, Loss: 0.876487
63m 16s
Epoch: 29, Loss: 0.847070
66m 13s
Epoch: 39, Loss: 0.826042
69m 9s
Epoch: 49, Loss: 0.819360
Accuracy: 0.500